In [2]:
# importing libraries
import pandas as pd
import matplotlib.pyplot as plt #graphical
import seaborn as sns #graphical
import numpy as np # mathematical essentials
import statsmodels.formula.api as smf # regression modeling
import statsmodels.formula.api as smf # regression modeling
from sklearn.model_selection import train_test_split # train/test split
from sklearn.linear_model import LinearRegression
import sklearn.linear_model # linear models


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


file = './datasets/Apprentice_Chef_Dataset.xlsx'

chef = pd.read_excel(io = file)

#creating a variable with coninuous variable from chef dataset

continuous_data = ['REVENUE','TOTAL_MEALS_ORDERED','UNIQUE_MEALS_PURCH',
                   'CONTACTS_W_CUSTOMER_SERVICE','PRODUCT_CATEGORIES_VIEWED',
                   'AVG_TIME_PER_SITE_VISIT','CANCELLATIONS_BEFORE_NOON',
                   'CANCELLATIONS_AFTER_NOON','MOBILE_LOGINS','PC_LOGINS',
                   'WEEKLY_PLAN','EARLY_DELIVERIES','LATE_DELIVERIES',
                   'AVG_PREP_VID_TIME','LARGEST_ORDER_SIZE','MEDIAN_MEAL_RATING',
                   'AVG_CLICKS_PER_VISIT','TOTAL_PHOTOS_VIEWED']


# log transfomation REVENUE and saving it in to the dataset
chef['log_REVENUE'] = np.log10(chef['REVENUE'])

# log transfomation Avg time per visit
chef['log_AVG_TIME_PER_SITE_VISIT'] = np.log10(chef['AVG_TIME_PER_SITE_VISIT'])

# log transfomation Avg Preparation video  and saving it in to the dataset
chef['log_AVG_PREP_VID_TIME'] = np.log10(chef['AVG_PREP_VID_TIME'])

# log transfomation Total Meals Ordered and saving it in to the dataset

chef['log_TOTAL_MEALS_ORDERED'] = np.log10(chef['TOTAL_MEALS_ORDERED'])

# log transfomation Unique Meals Purch and saving it in to the dataset

chef['log_UNIQUE_MEALS_PURCH'] = np.log10(chef['UNIQUE_MEALS_PURCH'])

# log transfomation Concat Customer Service Purch and saving it in to the dataset

chef['log_CONTACTS_W_CUSTOMER_SERVICE'] = np.log10(chef['CONTACTS_W_CUSTOMER_SERVICE'])

# instantiating an imputation value 
fill = 0 

# imputing 'FAMILY NAME'
chef['FAMILY_NAME'] = chef['FAMILY_NAME'].fillna(fill)

#counting the number of zeros 

CANCELATION_BEFORE_NOON_ZERO = len(chef['CANCELLATIONS_BEFORE_NOON'][chef['CANCELLATIONS_BEFORE_NOON']==0])
CANCELATION_AFTER_NOON_ZERO = len(chef['CANCELLATIONS_AFTER_NOON'][chef['CANCELLATIONS_AFTER_NOON']==0])
MOBILE_LOGINS = len(chef['MOBILE_LOGINS'][chef['MOBILE_LOGINS']==0])
WEEKLY_PLAN_ZERO = len(chef['WEEKLY_PLAN'][chef['WEEKLY_PLAN']==0])
EARLY_DELIVERIES_ZERO = len(chef['EARLY_DELIVERIES'][chef['EARLY_DELIVERIES']==0])
LATE_DELIVERIES_ZERO = len(chef['LATE_DELIVERIES'][chef['LATE_DELIVERIES']==0])
MASTER_CLASSES_ATTENDED_ZERO = len(chef['MASTER_CLASSES_ATTENDED'][chef['MASTER_CLASSES_ATTENDED']==0])
TOTAL_PHOTOS_VIEWED_ZERO = len(chef['TOTAL_PHOTOS_VIEWED'][chef['TOTAL_PHOTOS_VIEWED']==0])
MOBILE_NUMBER_O = len(chef['MOBILE_NUMBER'][chef['MOBILE_NUMBER']==0])


# creatinga varibales 1 - 0 
chef['Canceled_before_noon'] = 0
chef['Canceled_after_noon'] = 0
chef['Mobile_Login'] = 0
chef['has_weekly_plan'] = 0
chef['Was_early_delivered'] = 0
chef['was_late_delivered'] = 0
chef['Attended_master_class'] = 0
chef['View_photos'] = 0
chef['MOBILE_NUMBER_O'] = 0




for index, value in chef.iterrows():
    
    # Total_Bsmt_SF
    if chef.loc[index, 'CANCELLATIONS_BEFORE_NOON'] > 0:
        chef.loc[index, 'Canceled_before_noon'] = 1
    
    if chef.loc[index, 'CANCELLATIONS_AFTER_NOON'] > 0:
        chef.loc[index, 'Canceled_after_noon'] = 1
        
    if chef.loc[index, 'MOBILE_LOGINS'] > 0:
        chef.loc[index, 'Mobile_Login'] = 1
            
        
    if chef.loc[index, 'WEEKLY_PLAN'] > 0:
        chef.loc[index, 'has_weekly_plan'] = 1
        
    if chef.loc[index, 'EARLY_DELIVERIES'] > 0:
        chef.loc[index, 'Was_early_delivered'] = 1
        
    if chef.loc[index, 'LATE_DELIVERIES'] > 0:
        chef.loc[index, 'was_late_delivered'] = 1
    
    if chef.loc[index, 'MASTER_CLASSES_ATTENDED'] > 0:
        chef.loc[index, 'Attended_master_class'] = 1
    
    if chef.loc[index, 'TOTAL_PHOTOS_VIEWED'] > 0:
        chef.loc[index, 'View_photos'] = 1
        
    if chef.loc[index, 'MOBILE_NUMBER'] > 0:
        chef.loc[index, 'MOBILE_NUMBER_O'] = 1
        
    if chef.loc[index, 'PACKAGE_LOCKER'] > 0:
        chef.loc[index, 'HAS_PACKAGE_LOCKER'] = 1
        
        

        
# Creating a Variable locker with Refrigeration

chef['PACKAGE_LOCKER_and_REF'] = chef['PACKAGE_LOCKER'] + chef['REFRIGERATED_LOCKER']
# creating a binary variable
chef['PACKAGE_LOCKER_and_REF'] = chef['PACKAGE_LOCKER_and_REF'] - 1


# Orders delivered not as Schedule
chef['Not_as_schedule'] = chef['EARLY_DELIVERIES'] +chef['LATE_DELIVERIES']

# Average time per visit on the phone

chef['Avg_time_mobile_phone'] = chef['AVG_TIME_PER_SITE_VISIT']*chef['MOBILE_NUMBER']

# total Cancelations
chef['Total_Cancel'] = chef['CANCELLATIONS_BEFORE_NOON'] + chef['CANCELLATIONS_AFTER_NOON']

# customer services if attended master class

chef['contact_if_attended_class'] = chef['Attended_master_class'] * chef['CONTACTS_W_CUSTOMER_SERVICE']
        
# rating if attended to master class 

chef['raiting_if_attended_class'] = chef['MEDIAN_MEAL_RATING'] * chef['Attended_master_class']

# rate if view photos 
chef['Rate_if_view_photos'] = chef['View_photos'] * chef['MEDIAN_MEAL_RATING']
    
# rate if order by mobile     
chef['Mobile_Orders'] = chef['TOTAL_MEALS_ORDERED'] * chef['Mobile_Login']
    
x_variables = [ 'log_TOTAL_MEALS_ORDERED' , 'TOTAL_MEALS_ORDERED' ,'log_UNIQUE_MEALS_PURCH', 'UNIQUE_MEALS_PURCH' ,   
                'log_AVG_PREP_VID_TIME',  'log_CONTACTS_W_CUSTOMER_SERVICE' ,                        
                'LARGEST_ORDER_SIZE' , 'MEDIAN_MEAL_RATING','Attended_master_class', 
               'View_photos' ,'log_AVG_TIME_PER_SITE_VISIT', 'Avg_time_mobile_phone',
               'TOTAL_PHOTOS_VIEWED','contact_if_attended_class', 'raiting_if_attended_class', 'Rate_if_view_photos',
              'PRODUCT_CATEGORIES_VIEWED']


chef_data   = chef.drop(['CROSS_SELL_SUCCESS',
                                                 'REVENUE',
                                                 'log_REVENUE', 'NAME', 'EMAIL', 'FIRST_NAME', 'FAMILY_NAME','HAS_PACKAGE_LOCKER'],
                               axis = 1)


# applying modelin scikit-learn

# preparing x-variables from the OLS model
ols_data = chef[x_variables] # pvalues usefull before 
            #subset 

# preparing response variable
chef_target = chef['REVENUE']



# FULL X-dataset (normal Y)
x_train_FULL, x_test_FULL, y_train_FULL, y_test_FULL = train_test_split(
            chef_data,     # x-variables
            chef_target,   # y-variable
            test_size = 0.25,
            random_state = 219) 





# INSTANTIATING a model object
ard_model = sklearn.linear_model.ARDRegression(normalize  = False)


# FITTING the training data
ard_fit = ard_model.fit(x_train_FULL, y_train_FULL)


# PREDICTING on new data
ard_pred = ard_fit.predict(x_test_FULL)


print('Training Score:', ard_model.score(x_train_FULL, y_train_FULL).round(2))
print('Testing Score :',  ard_model.score(x_test_FULL, y_test_FULL).round(2))


# saving scoring data for future use
ard_train_score = ard_model.score(x_train_FULL, y_train_FULL).round(2)
ard_test_score  = ard_model.score(x_test_FULL, y_test_FULL).round(2)





# zipping each feature name to its coefficient
ard_model_values = zip(chef_data.columns, ard_fit.coef_.round(decimals = 5))


# setting up a placeholder list to store model features
ard_model_lst = [('intercept', ard_fit.intercept_.round(decimals = 2))]


# printing out each feature-coefficient pair one by one
for val in ard_model_values:
    ard_model_lst.append(val)

# printing out each feature-coefficient pair one by one
for feature, coefficient in ard_model_lst:
        
        if coefficient == 0:
            ard_model_lst.remove((feature, coefficient))
            
            
# saving prediction and deviatoin in a dataset

prediction_results = pd.DataFrame(data = {
    'Original Sale Prices' : y_test_FULL,
    'ARD Predictions'      : ard_pred.round(decimals = 2),
    'ARD Deviation'        : ard_pred.round(decimals = 2) - y_test_FULL,
    })

prediction_results.to_excel(excel_writer = 'linear_ARD_model_predictions.xlsx',
                            index = False)

# saving the model in a dataset

prediction_results = pd.DataFrame(data = {
    'Original Sale Prices' : y_test_FULL,
    'ARD Predictions'      : ard_pred.round(decimals = 2),
    'ARD Deviation'        : ard_pred.round(decimals = 2) - y_test_FULL,
    })





ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Home\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-17e594f1df84>", line 196, in <module>
    ard_fit = ard_model.fit(x_train_FULL, y_train_FULL)
  File "C:\Users\Home\Anaconda3\lib\site-packages\sklearn\linear_model\bayes.py", line 562, in fit
    sigma_ = update_sigma(X, alpha_, lambda_, keep_lambda, n_samples)
  File "C:\Users\Home\Anaconda3\lib\site-packages\sklearn\linear_model\bayes.py", line 547, in update_sigma
    X[:, keep_lambda].T))
  File "C:\Users\Home\Anaconda3\lib\site-packages\sklearn\externals\_scipy_linalg.py", line 99, in pinvh
    s, u = decomp.eigh(a, lower=lower, check_finite=False)
  File "C:\Users\Home\Anaconda3\lib\site-packages\scipy\linalg\decomp.py", line 432, in eigh
    iu=a1.shape[0], overwrite_a=overwrite_a)
KeyboardInterrupt

During handling of the above exception, another exception oc

KeyboardInterrupt: 

In [34]:
performance = pd.read_excel('linear_ARD_model_performance.xlsx')
prediction = pd.read_excel('linear_ARD_model_predictions.xlsx')
                            

performance.head(1)

                        

,Model Type,Training,Testing,Train-Test Gap,Model Size,Model
0,ARD,0.72,0.76,0.04,45,"[('intercept', -727.58), ('TOTAL_MEALS_ORDERED..."


In [37]:
prediction.head(10)

,Original Sale Prices,ARD Predictions,ARD Deviation
0,1000.0,766.34,-233.66
1,645.0,476.68,-168.32
2,1740.0,2130.03,390.03
3,2502.5,1907.08,-595.42
4,4260.0,2861.23,-1398.77
5,1050.0,847.46,-202.54
6,3750.0,2409.07,-1340.93
7,1190.0,741.72,-448.28
8,4790.0,4648.55,-141.45
9,5550.0,3849.63,-1700.37
